In [ ]:
!pip install transformers datasets spacy nltk rouge-score
!python -m spacy download en_core_web_sm

In [ ]:
import re
import nltk
import spacy
from heapq import nlargest
from transformers import pipeline
from datasets import load_dataset
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize

def preprocess_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    return text

In [ ]:
def extractive_summary(text, num_sentences=3):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    sentence_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.is_stop == False and word.is_alpha:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + 1
    summary = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return ' '.join([str(s) for s in summary])


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summary(text):
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']


In [ ]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
sample = dataset['test'][0]
print("Original Article:\n", sample['article'])
print("Reference Summary:\n", sample['highlights'])


In [ ]:
text = preprocess_text(sample['article'])

print("\nExtractive Summary:\n", extractive_summary(text))
print("\nAbstractive Summary:\n", abstractive_summary(text))
